In [1]:
import json
import numpy as np

def load_transforms(json_path):
    with open(json_path, 'r') as f:
        transforms = json.load(f)
    return transforms

def camera_center_from_transform(transform_matrix):
    """
    Given a 4x4 camera-to-world transform matrix (in standard computer-graphics convention),
    the camera center in world space is simply T @ [0,0,0,1].
    """
    transform_matrix = np.array(transform_matrix)
    # Multiply the matrix by the origin in camera coordinates
    origin_cam_space = np.array([0, 0, 0, 1], dtype=np.float32)
    center_world_space = transform_matrix @ origin_cam_space
    return center_world_space[:3]  # Extract x, y, z

def analyze_camera_positions(transforms_json):
    # Load the entire dictionary from transforms.json
    camera_transforms = load_transforms(transforms_json)
    
    # We will gather camera centers and look at their distances from the origin
    camera_positions = []
    
    for frame in camera_transforms["frames"]:
        transform_matrix = frame["transform_matrix"]
        center_world = camera_center_from_transform(transform_matrix)
        camera_positions.append(center_world)
    
    camera_positions = np.array(camera_positions)
    
    # Distances of each camera center from the world origin (0,0,0)
    distances = np.linalg.norm(camera_positions, axis=1)
    
    # Basic stats
    min_dist = distances.min()
    max_dist = distances.max()
    avg_dist = distances.mean()
    
    print(f"Number of frames: {len(camera_positions)}")
    print("Camera centers (world space) shape:", camera_positions.shape)
    print(f"Minimum distance from origin: {min_dist:.4f}")
    print(f"Maximum distance from origin: {max_dist:.4f}")
    print(f"Average distance from origin: {avg_dist:.4f}")
    
    # A simple heuristic for near/far could be around:
    near_plane = max(0.1, min_dist * 0.8)
    far_plane  = max_dist * 1.5
    
    print("Suggested near plane:", near_plane)
    print("Suggested far plane:", far_plane)


In [3]:
cam_dist = 1.8
transforms_json_path = f"data/ecb91f433f144a7798724890f0528b23_cam_{cam_dist}/transforms.json"
analyze_camera_positions(transforms_json_path)

Number of frames: 100
Camera centers (world space) shape: (100, 3)
Minimum distance from origin: 0.9000
Maximum distance from origin: 0.9000
Average distance from origin: 0.9000
Suggested near plane: 0.7199999336738787
Suggested far plane: 1.3500000528186837


In [4]:
cam_dist = 1.2
transforms_json_path = f"data/ecb91f433f144a7798724890f0528b23_cam_{cam_dist}/transforms.json"
analyze_camera_positions(transforms_json_path)

Number of frames: 100
Camera centers (world space) shape: (100, 3)
Minimum distance from origin: 0.6000
Maximum distance from origin: 0.6000
Average distance from origin: 0.6000
Suggested near plane: 0.47999999570684554
Suggested far plane: 0.9000000914054104


In [ ]:
actual_planes = [2, 6]